# A simple model implementation


In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

from seapopym.configuration.no_transport.parameter import ForcingParameters, FunctionalGroups, NoTransportParameters
from seapopym.configuration.parameters.parameter_environment import (
    ChunkParameter,
    ClientParameter,
    EnvironmentParameter,
    OutputParameter,
    PreProductionParameter,
)
from seapopym.configuration.parameters.parameter_forcing import ForcingUnit
from seapopym.configuration.parameters.parameter_functional_group import (
    FunctionalGroupUnit,
    FunctionalGroupUnitMigratoryParameters,
    FunctionalGroupUnitRelationParameters,
)
from seapopym.logging.custom_logger import set_critical, set_debug, set_error, set_verbose, set_warning
from seapopym.model.no_transport_model import NoTransportModel
from seapopym.standard.units import StandardUnitsLabels

In [ ]:
path_forcing = (
    "/Users/ash/Documents/Workspaces/Data/phd/cmems_mod_glo_bgc_my_0.083deg-lmtl-Fphy_PT1D-i_1709010880580.nc"
)
path_bio = "/Users/ash/Documents/Workspaces/Data/phd/cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i_1709007157635.nc"

In [ ]:
path_forcing = (
    "/Users/ash/Documents/Workspaces/Data/phd/cmems_mod_glo_bgc_my_0.083deg-lmtl-Fphy_PT1D-i_1709010880580.nc"
)
forcing = xr.open_dataset(path_forcing)
forcing["T"].attrs["units"] = StandardUnitsLabels.temperature.units
forcing

In [ ]:
path_bio = "/Users/ash/Documents/Workspaces/Data/phd/cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i_1709007157635.nc"
bio = xr.open_dataset(path_bio)
bio

> Set the verbosity of the program.


In [ ]:
MODE = 4

# ------------ #
if MODE == 1:
    set_debug()
elif MODE == 2:
    set_verbose()
elif MODE == 3:
    set_warning()
elif MODE == 4:
    set_error()
    warnings.filterwarnings("ignore")
else:
    set_critical()
    warnings.filterwarnings("ignore")

## Initialize the functional groups


In [ ]:
f_groups = []

day_layer = 1
night_layer = 1
energy_transfert = 0.1668
tr_max = 10.38
tr_rate = -0.11
inv_lambda_max = 150
inv_lambda_rate = 0.15

cohort_timesteps = [1] * np.ceil(tr_max).astype(int)

fg_f_param = FunctionalGroupUnitRelationParameters(
    inv_lambda_max=inv_lambda_max,
    inv_lambda_rate=inv_lambda_rate,
    temperature_recruitment_rate=tr_rate,
    cohorts_timesteps=cohort_timesteps,
    temperature_recruitment_max=tr_max,
)
fg_m_param = FunctionalGroupUnitMigratoryParameters(
    day_layer=day_layer,
    night_layer=night_layer,
)
f_groups.append(
    FunctionalGroupUnit(
        name=f"D{day_layer}N{night_layer}",
        migratory_type=fg_m_param,
        functional_type=fg_f_param,
        energy_transfert=energy_transfert,
    )
)

f_groups = FunctionalGroups(functional_groups=f_groups)

In [ ]:
p_param = ForcingParameters(
    temperature=ForcingUnit(forcing=forcing["T"], resolution=0.08333),
    primary_production=ForcingUnit(forcing=bio["npp"], resolution=0.08333),
    initial_condition_biomass=None,
    initial_condition_production=None,
)

parameters = NoTransportParameters(
    functional_groups_parameters=f_groups,
    forcing_parameters=p_param,
    environment_parameters=EnvironmentParameter(
        chunk=ChunkParameter(functional_group=1, latitude=12),
        client=ClientParameter(n_workers=1, threads_per_worker=1, memory_limit="6GB"),
        output=OutputParameter(pre_production=PreProductionParameter(timestamps="all")),
    ),
)

## Initialize the model


In [ ]:
my_class = NoTransportModel(configuration=parameters)
my_class.generate_configuration()
my_class.state

In [ ]:
my_class.initialize()
my_class.client

## Compute pre-production

This part is computed before the production step. All part of the process that can be pre-computed in parallele should be done here.


In [ ]:
my_class.pre_production()

## Compute production

This step is time dependent.


In [ ]:
my_class.production()

## Here we compute the output of the model


In [ ]:
my_class.post_production()

## Plotting the results


In [ ]:
biomass_seapopym = my_class.state.biomass.load()
biomass_lmtl = bio.zooc.pint.quantify().pint.to(StandardUnitsLabels.biomass.units).pint.dequantify()

In [ ]:
(biomass_seapopym.cf.mean(("X", "Y"))).cf.plot.line(x="T", hue="functional_group", label="Zoo Seapopym")
(biomass_lmtl.cf.mean(("X", "Y"))).cf.plot.line(x="T", label="Zoo LMTL")
plt.legend()
plt.show()